In [229]:
from openfermion import get_sparse_operator
import numpy as np

from tangelo.linq import Circuit, Gate, get_backend
from tangelo.toolboxes.ansatz_generator.ansatz_utils import trotterize
from tangelo.molecule_library import mol_H2_sto3g
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping
from tangelo.toolboxes.qubit_mappings.statevector_mapping import get_reference_circuit

fe_op = mol_H2_sto3g.fermionic_hamiltonian
qu_op = fermion_to_qubit_mapping(fe_op, "scbk", n_spinorbitals=4, n_electrons=2)

ham = get_sparse_operator(qu_op).toarray()

hf_circ = get_reference_circuit(4, 2, "scbk", True, 0)

eigs, vecs = np.linalg.eigh(ham)
sim = get_backend("cirq")
_, hf_state = sim.simulate(hf_circ, return_statevector=True)




In [273]:
w_circ = Circuit([Gate("RY", 2, parameter=-np.pi/2), Gate("PHASE", 2, parameter = np.pi/2)])

def pite_circuit(qubit_op, tau, theta):
    circuit = (Circuit([Gate("H", 2)])+w_circ+
               Circuit([Gate("X", 2)]) + trotterize(qubit_op, tau, control=2, trotter_order=4, n_trotter_steps=4)+ Circuit([Gate("X", 2)])+
               trotterize(qubit_op, tau, control=2, trotter_order=4, n_trotter_steps=4).inverse()+
               Circuit([Gate("RZ", 2, parameter=-2*theta)])+
               w_circ.inverse()+
               Circuit([Gate("MEASURE", 2)]))
    return circuit


In [278]:
n_steps = 6
dmeas = "0"*n_steps
m0 = 0.8
kappa = np.sign(m0-1/np.sqrt(2))


theta0 = kappa*np.arccos((m0+np.sqrt(1-m0**2))/np.sqrt(2))
s1 = m0/np.sqrt(1-m0**2)

print(theta0, s1)


pite = pite_circuit(qubit_op=qu_op, tau=s1*9, theta=-theta0)*n_steps
f, sv = sim.simulate(hf_circ+pite, desired_meas_result=dmeas, return_statevector=True)
print(sv)
sv = np.reshape(sv, (4, 2))[:, 0]


0.14189705460416518 1.3333333333333337
[-4.77838847e-02+1.24445325e-01j  0.00000000e+00+0.00000000e+00j
  3.79629188e-16+7.04862515e-16j  0.00000000e+00+0.00000000e+00j
 -7.83046051e-18-1.01797739e-16j -0.00000000e+00+0.00000000e+00j
  4.50366501e-01-8.82836381e-01j -0.00000000e+00+0.00000000e+00j]


In [279]:
print(f'initial overlap = {abs(np.dot(hf_state, vecs[:, 0]))}')
print(f'final overlap = {abs(np.dot(sv.conj(), vecs[:, 0]))}')
print(sv)
print(vecs[:,0])


initial overlap = 0.9936146058054715
final overlap = 0.9997053728327631
[-4.77838847e-02+1.24445325e-01j  3.79629188e-16+7.04862515e-16j
 -7.83046051e-18-1.01797739e-16j  4.50366501e-01-8.82836381e-01j]
[ 0.11282737+0.j  0.        +0.j  0.        +0.j -0.99361461+0.j]
